# Múltiples variables almacenadas en una columna

Este cuaderno muestra cómo se pueden aislar varias variables almacenadas en la misma columna.

## "Housekeeping"

In [1]:
import pandas as pd

## Ejemplo: Tuberculosis

### Cargando la Data

Seleccionamos las mismas columnas que en el *paper* y le asignamos el nombre correspondiente.

In [2]:
columns = [
    "iso2",
    "year",
    "new_sp_m014",
    "new_sp_m1524",
    "new_sp_m2534",
    "new_sp_m3544",
    "new_sp_m4554",
    "new_sp_m5564",
    "new_sp_m65",
    "new_sp_mu",
    "new_sp_f014",
    "new_sp_f1524",
    "new_sp_f2534",
    "new_sp_f3544",
    "new_sp_f4554",
    "new_sp_f5564",
    "new_sp_f65",
    "new_sp_fu",
]
tb = pd.read_csv("data/tb.csv", usecols=columns)

rename = {column: column[7:] for column in columns if column.startswith("new_sp_")}
rename = {"iso2": "country", **rename}
tb = tb.rename(columns=rename)

### Messy Data
Se asume que los datos se proporcionan de la siguiente manera. Excepto por las columnas `"country"` y `"year"`, las columnas restantes son en realidad realizaciones conjuntas de dos variables `"sex"` y `"age"`.

The data are assumed to be provided as below. Except for the `"country"` and `"year"` columns, the remaining columns are actually joint realizations of two variables `"sex"` and `"age"`.

In [3]:
tb[(tb["year"] == 2000)].head(10)

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014,f1524,f2534,f3544,f4554,f5564,f65,fu
10,AD,2000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,AE,2000,2.0,4.0,4.0,6.0,5.0,12.0,10.0,NaN,3.0,16.0,1.0,3.0,0.0,0.0,4.0,NaN
60,AF,2000,52.0,228.0,183.0,149.0,129.0,94.0,80.0,NaN,93.0,414.0,565.0,339.0,205.0,99.0,36.0,NaN
87,AG,2000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,NaN
136,AL,2000,2.0,19.0,21.0,14.0,24.0,19.0,16.0,NaN,3.0,11.0,10.0,8.0,8.0,5.0,11.0,NaN
165,AM,2000,2.0,152.0,130.0,131.0,63.0,26.0,21.0,NaN,1.0,24.0,27.0,24.0,8.0,8.0,4.0,NaN
178,AN,2000,0.0,0.0,1.0,2.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0,1.0,0.0,NaN
207,AO,2000,186.0,999.0,1003.0,912.0,482.0,312.0,194.0,NaN,247.0,1142.0,1091.0,844.0,417.0,200.0,120.0,NaN
236,AR,2000,97.0,278.0,594.0,402.0,419.0,368.0,330.0,NaN,121.0,544.0,479.0,262.0,230.0,179.0,216.0,NaN
265,AS,2000,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


### *Molten* Data

Al igual que en el notebook anterior, la función [pd.melt()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html) se puede usar para despivotear las columnas. Como antes, pandas mantiene las filas de las columnas con *missing data*, los que se descartan. Luego, sin más *missing data*, el tipo de datos de la columna se convierte como `int`. Luego, el conjunto de datos *fundido* resultante es ordenado como en el paper.


In [4]:
molten_tb = pd.melt(
    tb, id_vars=["country", "year"], var_name="column", value_name="cases"
)
molten_tb = molten_tb[molten_tb["cases"].notnull()]
molten_tb["cases"] = molten_tb["cases"].astype(int)
molten_tb = molten_tb.sort_values(["country", "year", "column"]).reset_index(drop=True)

In [5]:
molten_tb[(molten_tb["year"] == 2000)].head(10)

,country,year,column,cases
49,AD,2000,m014,0
50,AD,2000,m1524,0
51,AD,2000,m2534,1
52,AD,2000,m3544,0
53,AD,2000,m4554,0
54,AD,2000,m5564,0
55,AD,2000,m65,0
165,AE,2000,f014,3
166,AE,2000,f1524,16
167,AE,2000,f2534,1


### Tidy Data
Usando el método [pd.Series.str.extract()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.extract.html) las dos variables son aisladas/separadas ("splitted"). Las etiquetas de `age` se renombran como en el documento.

In [6]:
tidy_tb = molten_tb[["country", "year", "cases"]]
tidy_tb[["sex", "age"]] = molten_tb["column"].str.extract(r"(f|m)(.*)")
tidy_tb["age"] = tidy_tb["age"].map(
    {
        "014": "0-14",
        "1524": "15-24",
        "2534": "25-34",
        "3544": "35-44",
        "4554": "45-54",
        "5564": "55-64",
        "65": "65+",
        "u": "unknown",
    }
)
tidy_tb = tidy_tb[["country", "year", "sex", "age", "cases"]]

In [7]:
tidy_tb[(tidy_tb["year"] == 2000)].head(10)

,country,year,sex,age,cases
49,AD,2000,m,0-14,0
50,AD,2000,m,15-24,0
51,AD,2000,m,25-34,1
52,AD,2000,m,35-44,0
53,AD,2000,m,45-54,0
54,AD,2000,m,55-64,0
55,AD,2000,m,65+,0
165,AE,2000,f,0-14,3
166,AE,2000,f,15-24,16
167,AE,2000,f,25-34,1
